In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Create claims dimensional table

In [0]:
%sql
-- creating a dim table with external location(ADLS) 
CREATE TABLE IF NOT EXISTS gold.dim_claims(
ClaimID STRING,
SRC_ClaimID STRING,
TransactionID STRING,
PatientID STRING,
EncounterID STRING,
ProviderID STRING,
DeptID STRING,
ServiceDate DATE,
ClaimDate DATE,
PayorID STRING,
ClaimAmount STRING,
PaidAmount STRING,
ClaimStatus STRING,
PayorType STRING,
Deductible STRING,
Coinsurance STRING,
Copay STRING,
datasource STRING
)
USING DELTA
LOCATION "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_claims";

In [0]:
%sql 
-- reset the table
truncate table gold.dim_claims;

In [0]:
%sql
-- finally inserting the record into it from the silver claims table
-- filtering by (is_quarantined = false and is_current = True)
INSERT INTO gold.dim_claims
SELECT 
ClaimID,
SRC_ClaimID,
TransactionID,
PatientID,
EncounterID,
ProviderID,
DeptID,
ServiceDate,
ClaimDate,
PayorID,
ClaimAmount,
PaidAmount,
ClaimStatus,
PayorType,
Deductible,
Coinsurance,
Copay,
datasource
from silver.claims
where
is_quarantined = false and is_current = True;


In [0]:
%sql
-- reading some records
select * from gold.dim_claims;